In [34]:
import pandas as pd
import numpy as np

In [57]:
df = pd.read_csv('tidy_2.csv')

## 새로운 칼럼 만들기

### 1) 대상인원 칼럼 만들기

In [58]:
df['대상인원'] = df['대상인원'].copy().str.extract(r'[\s|외|총|등](\d+)\s?[명|인]?')
# 대상인원이 없는 경우 운전기사 회식, 격려금 지급 등 맛집과 상관없는 경우가 많으니 삭제
df = df.dropna(axis=0, subset=['대상인원'], how='all')
df['대상인원'] = df['대상인원'].copy().astype(int)

### 2) 집행지명으로 그룹화하기

In [71]:
df_group = df.groupby('집행지명')\
            ['집행지명', 'cnt', '집행금액', '대상인원']\
            .sum().reset_index()
df_group.head(3)

,집행지명,cnt,집행금액,대상인원
0,(사)탈북여성지원,1,15500.0,3
1,(사)탈북여성지원지에프에스우물가,1,25000.0,7
2,(주)강가,1,160000.0,7


In [72]:
# 회당, 인당 금액을 구해보자. 
df_group['회당집행금액'] = df_group['집행금액'] / df_group['cnt']
df_group['인당집행금액'] = df_group['집행금액'] / df_group['대상인원']
df_group.head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액
0,(사)탈북여성지원,1,15500.0,3,15500.0,5166.666667
1,(사)탈북여성지원지에프에스우물가,1,25000.0,7,25000.0,3571.428571
2,(주)강가,1,160000.0,7,160000.0,22857.142857


## 카테고리별 맛집

### 1. 가장 자주 간 식당 (전체 공무원)

In [75]:
df_group.sort_values(by='cnt', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액
2079,우도일식,307,41795800.0,1936,136142.671010,21588.739669
2519,참숯골,261,33933380.0,1879,130012.950192,18059.276211
416,구이구이,248,23512200.0,1335,94807.258065,17612.134831


### 2. 가장 돈을 많이 쓴 식당 (전체 공무원)

In [76]:
df_group.sort_values(by='집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액
2079,우도일식,307,41795800.0,1936,136142.671010,21588.739669
2519,참숯골,261,33933380.0,1879,130012.950192,18059.276211
801,동해일식,168,29451000.0,1206,175303.571429,24420.398010


### 3. 회당 집행금액이 가장 많은 식당
(10회 이상 방문, 전체 공무원)

In [87]:
df_group_5_up = df_group[df_group['cnt']>10]
df_group_5_up.sort_values(by='회당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액
677,더미,62,19996000.0,778,322516.129032,25701.799486
2482,진미족발,26,7997000.0,430,307576.923077,18597.674419
2938,한국회관,13,3618000.0,194,278307.692308,18649.484536


### 4. 인당 집행금액이 가장 많은 식당
(10회 이상 방문, 전체 공무원)

In [88]:
df_group_5_up = df_group[df_group['cnt']>10]
df_group_5_up.sort_values(by='인당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액
1100,몽중헌,15,2581600.0,94,172106.666667,27463.829787
1613,송원복집,17,2434000.0,89,143176.470588,27348.314607
780,동원브이아이피참치,19,3599000.0,134,189421.052632,26858.208955


## 부서별로 살펴보기

In [102]:
# 부서별로 묶기
df_divs = df.groupby('1소속부서')['cnt', '대상인원', '집행금액']\
        .sum().reset_index()
df_divs.head(3)

,1소속부서,cnt,대상인원,집행금액
0,감사위원회,435,2483,44830570.0
1,관광체육국,892,4762,83613736.0
2,기술심사담당관,83,702,10756895.0


In [107]:
# 인당/회당 금액 내기
df_divs['인당집행금액'] = df_divs['집행금액'] / df_divs['대상인원']
df_divs['회당집행금액'] = df_divs['집행금액'] / df_divs['cnt']

### 1. 가장 회식을 자주 한 부서

In [120]:
df_divs.sort_values(by='cnt', ascending=False).head(3)

,1소속부서,cnt,대상인원,집행금액,인당집행금액,회당집행금액
25,행정국,1335,8073,171576540.0,21253.132664,128521.752809
15,시민소통기획관,1320,9327,199958077.0,21438.627319,151483.391667
4,기후환경본부,990,5594,97037616.0,17346.731498,98017.793939


### 2. 가장 돈을 많이 쓴 부서

In [121]:
df_divs.sort_values(by='집행금액', ascending=False).head(3)

,1소속부서,cnt,대상인원,집행금액,인당집행금액,회당집행금액
15,시민소통기획관,1320,9327,199958077.0,21438.627319,151483.391667
25,행정국,1335,8073,171576540.0,21253.132664,128521.752809
16,안전총괄본부,887,6601,138440157.0,20972.603696,156076.839910


관광체육국은 자주 회식을 하는 것에 비해 돈을 많이 쓰지는 않으니 싸고 맛있는 집을 많이 가겠군! 라고 추론해보았다. 

### 3. 가장 비싼 것을 먹는 부서

In [124]:
df_divs.sort_values(by='인당집행금액', ascending=False).head(5)

,1소속부서,cnt,대상인원,집행금액,인당집행금액,회당집행금액
20,정무부시장,753,4685,115240050.0,24597.662753,153041.235060
23,행정1부시장,234,1832,44650630.0,24372.614629,190814.658120
18,일자리노동정책관,549,2626,60448165.0,23019.103199,110105.947177
5,대변인,976,5410,121820665.0,22517.682994,124816.255123
10,비상기획관,288,1363,30663150.0,22496.808511,106469.270833


### 4. 가장 싼 것을 먹는 부서

In [125]:
df_divs.sort_values(by='인당집행금액', ascending=True).head(5)

,1소속부서,cnt,대상인원,집행금액,인당집행금액,회당집행금액
9,복지본부,612,3328,50729070.0,15243.109976,82890.637255
2,기술심사담당관,83,702,10756895.0,15323.212251,129601.144578
6,도시공간개선단,258,1445,22193952.0,15359.136332,86023.069767
17,여성가족정책실,687,4613,75427795.0,16351.137004,109793.005822
24,행정2부시장,402,8155,135462850.0,16611.017781,336972.263682


행정1부시장에 비해 행정2부시장이 가성비맛집을 많이 갈 확률이 높다!